# Best Predictors of the World Series Winners?
Of the teams that make the playoffs, is there a way to predict who the world series winner will be?

Features - usually more informative to do ratios between values instead of just straight up values.
* how many times in playoffs in last 5 years
* Run differential? 
* salary
* all star/cy young?
* win percentage in second half of season
* hot pitcher(s) - K/BB, FIP, WHIP, IP/GP, ERA of pitcher

Different approaches:  
* Maybe predict who will be in the world series final instead? Maybe it's too hard to know who will actually win. Rank teams in various categories relative to the other teams in your division.
* Maybe some world series come down to chance, whereas others are more deterministic. For example, if the world series champions had game 7's in each series, then a slight perturbation to this system causes them not to ultimately win the WS. However maybe other world series are much more clear in who will win.  
* Probably need a more complicated model, i.e. see the playoff bracket and predict the outcomes, having weights at each node. Probably a job for the Neural Network.

In [55]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import plotly
import plotly.plotly as py
import plotly.graph_objs as go
plotly.offline.init_notebook_mode()

## Collect Data
Collect, clean, organize data.

In [56]:
#load data and grab columns of interest
team = pd.read_csv("csv/team.csv")

del_columns = ["div_id","name","team_id_lahman45","franchise_id","team_id_retro","team_id_br","ppf","bpf","park"]
team.drop(del_columns, axis=1, inplace=True)

year_cutoff = 1969
team = team[team["year"]>=year_cutoff]

In [57]:
team.loc[team["ws_win"] == "Y", "ws_win"] = 1
team.loc[team["ws_win"] == "N", "ws_win"] = 0
team.loc[pd.isnull(team["ws_win"]), "ws_win"] = 0

team.loc[team["wc_win"] == "Y", "wc_win"] = 1
team.loc[team["wc_win"] == "N", "wc_win"] = 0
team.loc[pd.isnull(team["wc_win"]), "wc_win"] = 0

team.loc[team["lg_win"] == "Y", "lg_win"] = 1
team.loc[team["lg_win"] == "N", "lg_win"] = 0
team.loc[pd.isnull(team["lg_win"]), "lg_win"] = 0

team.loc[team["div_win"] == "Y", "div_win"] = 1
team.loc[team["div_win"] == "N", "div_win"] = 0
team.loc[pd.isnull(team["div_win"]), "div_win"] = 0

team["postseason"] = 0
team.loc[(team["div_win"]==1) | (team["wc_win"]==1),"postseason"] = 1

In [58]:
#Looks like they only made divisions in 1969. 
team.head()

,year,league_id,team_id,rank,g,ghome,w,l,div_win,wc_win,...,ipouts,ha,hra,bba,soa,e,dp,fp,attendance,postseason
1517,1969,NL,ATL,1,162,81.0,93,69,1,0,...,4335,1334,144,438,893,115,114.0,0.98,1458320.0,1
1518,1969,AL,BAL,1,162,81.0,109,53,1,0,...,4419,1194,117,498,897,101,145.0,0.98,1062069.0,1
1519,1969,AL,BOS,3,162,81.0,87,75,0,0,...,4398,1423,155,685,935,157,178.0,0.97,1833246.0,0
1520,1969,AL,CAL,3,163,81.0,71,91,0,0,...,4314,1294,126,517,885,135,164.0,0.97,758388.0,0
1521,1969,AL,CHA,5,162,81.0,68,94,0,0,...,4311,1470,146,564,810,122,163.0,0.98,589546.0,0


## Feature Engineering
Especially for this type of problem, feature selection is by far the most important part. Using the standard columns in this dataframe yields a terrible fit using a random forest. 

In [59]:
team_names = team["team_id"].unique()

In [60]:
#Number of playoff appearances in the last 5 years
team["5yrpost"] = 0
for t in team_names:
    team_t = team[team["team_id"]==t]
    team.loc[team["team_id"]==t,"5yrpost"] = team_t["postseason"].rolling(window=5,min_periods=2).sum()
team.loc[team["5yrpost"].isnull(),"5yrpost"] = 0

In [61]:
#Ratios - X for / X against
team["r_r"] = team["r"]/team["er"]  #runs
team["h_r"] = team["h"]/team["ha"]  #hits

### Pitching Features

In [62]:
pitching = pd.read_csv("csv/pitching.csv")
pitching = pitching[pitching["year"]>=year_cutoff]

In [63]:
#whip - anything below 1.1 is considered great, below 1 is outstanding. Check for # ipouts though!!
#SO/BB - ratio of strikeouts to walks. 
#prevent inf values
pitching.loc[pitching["ipouts"]==0, "ipouts"] = 1
pitching.loc[pitching["bb"]==0, "bb"] = 1

pitching["whip"] = (pitching["bb"] - pitching["ibb"] + pitching["h"]) / (pitching["ipouts"]/3.)
pitching["so/bb"] = pitching["so"]/pitching["bb"]
pitching["ip/g"] = pitching["ipouts"]/(3*pitching["g"])
pitching.loc[pd.isnull(pitching["ip/g"]),"ip/g"] = 0

pitching["whip"] = (pitching["whip"] - pitching["whip"].mean())/pitching["whip"].std()
pitching["so/bb"] = (pitching["so/bb"] - pitching["so/bb"].mean())/pitching["so/bb"].std()
#pitching["ip/g"] = (pitching["ip/g"] - pitching["ip/g"].mean())/pitching["ip/g"].std()

pitching["hot_pitcher"] = 0
sdfm_thresh = 1              #number of standard deviations from the mean to define a "hot pitcher"
pitching.loc[(pitching["whip"]<= -sdfm_thresh)|(pitching["so/bb"]>=sdfm_thresh)|(pitching["ip/g"]>=7),"hot_pitcher"]=1
pitching.loc[(pitching["w"]<10) & (pitching["sv"]<15),"hot_pitcher"] = 0       #IP*3 > 100 and g > 10
#pitching[pitching["ip/g"]>7]

In [64]:
#Convert pitching stats into team stats
pitching=pitching.groupby(["team_id","year"]).sum()
pitching = pitching.reset_index()
team["n_hp"] = 0

for t in team_names:
    team.loc[team["team_id"]==t,"n_hp"] = pitching.loc[pitching["team_id"]==t,"hot_pitcher"].values

In [65]:
team.loc[:,["year","team_id","w","era","sv","postseason","5yrpost","n_hp","h_r","r_r"]].head()

,year,team_id,w,era,sv,postseason,5yrpost,n_hp,h_r,r_r
1517,1969,ATL,93,3.53,42,1,0.0,1,1.057721,1.218695
1518,1969,BAL,109,2.83,36,1,0.0,1,1.226968,1.682505
1519,1969,BOS,87,3.92,41,0,0.0,1,0.970485,1.162754
1520,1969,CAL,71,3.54,39,0,0.0,0,0.943586,0.932862
1521,1969,CHA,68,4.21,25,0,0.0,0,0.915646,0.930060


The most interesting question to ask is "Out of the teams that made the playoffs, who is most likely to win the world series?". So, we need to filter out all the teams that didn't make the playoffs each year. 

In [66]:
team = team.loc[team["postseason"]==1]

Grab only the features we consider most predictive of world series performance.

## Pre-processing

In [67]:
Xcolumns = ["w","r","sv","era","fp","5yrpost","n_hp","h_r","r_r"]
#Xcolumns = ["w","n_hp","h_r","r_r","5yrpost"]
y = team["ws_win"].astype(int)
X = team[Xcolumns]

Finally, some ML methods require that all the features are scaled, so we will use the Xs dataframe for scaled values of all our features.

In [16]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X)
Xs = scaler.transform(X)    #scale data

### Principal Component Analysis (PCA)
Dimensionality reduction is always key to an efficient algorithm. It is also very useful to determine which features have the greatest importance.

Each principal component is a linear combination of the original features:  
$PC^i = B^i_1X_1 + B^i_2X_2 + ... + B^i_nX_n$  

where $B_j$ are the weights and $X_i$ are the features. Thus the (absolute value of the) weights correspond to feature importance for constructing the component of maximal variance. As can be seen below, it looks like there is no single dominating feature.

In [17]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(Xs)                 #get PCA of scaled data
pd.DataFrame(pca.components_.T, index=X.columns)

,0,1
w,0.482786,-0.343840
r,0.162682,-0.601172
sv,0.074016,-0.366596
era,-0.342424,-0.508295
fp,-0.160620,-0.255035
5yrpost,0.095633,-0.123802
n_hp,0.122726,0.194373
h_r,0.467098,0.015233
r_r,0.591213,0.096053


It's always good to actually look at the data and see if you can personally recognize any patterns. There's a good chance that if you can't recognize anything, the machine learning won't either.

In [84]:
arr = X[y==1]
arr2 = X[y==0]
c = ["w","5yrpost","era"]

trace0 = go.Scatter3d(
    x=arr[c[0]],
    y=arr[c[1]],
    z=arr[c[2]],
    mode='markers',
    name='ws winners',
    marker=dict(
        size=8,
        #color=y,             # set color to an array/list of desired values
        #colorscale='RdGy',   # choose a colorscale
        opacity=0.8
    )
)

trace1 = go.Scatter3d(
    x=arr2[c[0]],
    y=arr2[c[1]],
    z=arr2[c[2]],
    mode='markers',
    name='ws losers',
    marker=dict(
        size=4,
        #color=y,             # set color to an array/list of desired values
        #colorscale='RdGy',   # choose a colorscale
        opacity=0.8
    )
)

data = [trace0,trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

## Data Analysis

In [16]:
from sklearn.cross_validation import train_test_split,cross_val_score
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
Xs_train, Xs_test, ys_train, ys_test = train_test_split(Xs, y, test_size=0.25, random_state=43)

### An important note about Train/Test/CV
Usually you split your data into 3 chunks: Train, test, and validation sets:  
1) Training set: The gold standard where you know the answers. Train your model with this set.  
2) Validation set: Here you also know the answers, and want to test the accuracy of your model using some kind of scoring system. This lets you know if you've overfit/underfit your model to the training set. From scikit-learn:   
*When evaluating different settings (“hyperparameters”) for estimators, such as the C setting that must be manually set for an SVM, there is still a risk of overfitting on the test set because the parameters can be tweaked until the estimator performs optimally. This way, knowledge about the test set can “leak” into the model and evaluation metrics no longer report on generalization performance. To solve this problem, yet another part of the dataset can be held out as a so-called “validation set”: training proceeds on the training set, after which evaluation is done on the validation set, and when the experiment seems to be successful, final evaluation can be done on the test set.*  
To avoid splitting your data endlessly leaving each set with very few data points, CV is introduced, merging the training and validation set into one and doing internal splitting. But you still need to reserve a final test set. The CV is done on the training set.  
3) Test set: This is the final test of your model to see how well it performs.  

Apparently, the K-fold CV score in general is the best method for determining your predictive accuracy.

ref: http://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-evaluating-estimator-performance

### SVM

In [49]:
#We learned from RF that accuracy score is not a good metric for predicting world series winners. AUC is better.
from sklearn.svm import SVC
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.grid_search import GridSearchCV
cv_s = StratifiedShuffleSplit(ys_train,  n_iter=10 , test_size=0.1, random_state=42)
clf=SVC()
param_grid = {'kernel':('linear', 'rbf','poly'), 'C':[1,10,20], 'degree':[2,3,5]}
CV_svm = GridSearchCV(n_jobs=-1, estimator=clf, scoring="roc_auc", param_grid=param_grid, cv=cv_s)
CV_svm.fit(Xs_train, ys_train)
print("Best Parameters from gridsearch: {%s} with a score of %0.4f" % (CV_svm.best_params_, CV_svm.best_score_))

Best Parameters from gridsearch: {{'kernel': 'poly', 'C': 20, 'degree': 2}} with a score of 0.5809


In [52]:
#might be good to find weights for the positive world series winners.
from sklearn import metrics
model_svm = CV_svm.best_estimator_
model_svm.score(Xs_test, ys_test)

0.87142857142857144

### Random Forest

In [40]:
from sklearn.ensemble import RandomForestClassifier
cv_s = StratifiedShuffleSplit(y_train,  n_iter=10 , test_size=0.1, random_state=42)
rfc = RandomForestClassifier(max_features= 'auto' ,n_estimators=50) 
param_grid = { 
        'n_estimators': [600],
        'max_features': ['sqrt'],
        'min_samples_leaf': [10]}
CV_rfc = GridSearchCV(n_jobs=-1, estimator=rfc, scoring="roc_auc", param_grid=param_grid, cv=cv_s)
CV_rfc.fit(X_train, y_train);
print("Best Parameters from gridsearch: {%s} with a score of %0.4f" % (CV_rfc.best_params_, CV_rfc.best_score_))

Best Parameters from gridsearch: {{'max_features': 'sqrt', 'n_estimators': 600, 'min_samples_leaf': 10}} with a score of 0.6490


In [51]:
from sklearn import metrics
model = CV_rfc.best_estimator_
y_pred = model.predict_proba(X_test) #probability that team0 wins (what Kaggle calls team 1, and wants for submission)
y_pred_acc = model.predict(X_test)   #firm class vote by all the trees, 0 or 1 in this case.
test_score = metrics.roc_auc_score(y_test, y_pred[:,1], average="weighted") #area under curve from prediction scores
test_score_acc = metrics.accuracy_score(y_test, y_pred_acc)
print("AUC score is {0}".format(test_score))
print("Accuracy is {0}".format(test_score_acc))

AUC score is 0.469953775039
Accuracy is 0.842857142857


In [ ]:
y_pred_acc

The accuracy score is not(!) a good indicator for this kind of problem since most of the entries will always be 0 (i.e. only 1 in 10 teams will win the world series)! Our y_pred_acc array is filled completely with 0's (i.e. not a single positive world series prediction) and we still get an 86% score. The random forest model sucks right now!

In [ ]:
print("Feature\t\tImportance\n")
for i in reversed(np.argsort(model.feature_importances_)):
    print("%s\t\t%f" % (X.columns[i], model.feature_importances_[i]))

# Extra

In [ ]:
#Some analysis tools 
#n, bins, patches = plt.hist(pitching["ipouts"].loc[pitching["hot_pitcher"]==1]/(3*pitching["g"].loc[pitching["hot_pitcher"]==1]), 50)

#pitching.loc[pitching["g"]==0, "g"] = 1

pitching.loc[pd.isnull(pitching["ip/g"])]

#pitching["ip/g"] = pitching["ipouts"]/(3*pitching["g"])
n, bins, patches = plt.hist(pitching["ip/g"], 50)
#plt.xlim([0,10])

#n, bins, patches = plt.hist(pitching["so_bb"], 50)
#plt.xlim([0,10])
#pitching["so_bb"].std()